# Image segmentation
ต่างกับ Object detection คือมีความละเอียดมากกว่าเพราะสามารถบอกว่าวัตถุนั้นคืออะไร ในระดับ Pixel ซึ่งจะระบายสีตามแต่ละ object ไม่ได้เป็นการตีกรอบเหมือน Object detection
เช่น Model SAM, SlimSAM (ใช้บนอุปกรณ์ที่ทรัพยากรจำกัดได้)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from transformers import pipeline

In [ ]:
model = 'Zigeng/SlimSAM-uniform-77'
seg_pipe = pipeline('mask-generation', model=model)

In [ ]:
!wget https://media.wired.com/photos/593256b42a990b06268a9e21/master/pass/traffic-jam-getty.jpg -O cars.jpg

In [ ]:
image = Image.open('cars.jpg')
plt.imshow(image)
plt.show()

In [ ]:
result = seg_pipe(image)
result

In [ ]:
result.keys()

In [ ]:
len(result['masks'])

In [ ]:
plt.imshow(result['masks'][0])
plt.show()
# การใช้งานจริงการใช้แค่ 1 masks อาจะไม่เพียงพอ อาจะต้องใข้ masks อื่นๆ ไป overrate เพิ่ม
# หรือวางทับบนภาพ original

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

def show_mask(mask, ax, random_color=False):
  if random_color:
    color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    # ต่อ array ของ random color กับ ค่า โปร่งใส
  else:
    color = np.array([30/255, 144/255, 255/255, 0.6])
  h,w = mask.shape[-2:]
  mask_image = mask.reshape(h,w,1) * color.reshape(1,1,-1)
  ax.imshow(mask_image)


In [ ]:
def show_pipe_masks_on_image(raw_image, outputs):
  plt.imshow(np.array(raw_image))
  ax = plt.gca()
  for mask in outputs['masks']:
    show_mask(mask, ax=ax, random_color=True)
  # plt.axis('off')
  plt.show()

In [ ]:
show_pipe_masks_on_image(image, result)

In [ ]:
# normally จะนำผลลัพธ์จาก mask generatation ไปเข้า classification ต่อเพื่อตีความหมาย

**ทำ Segmentation แบบกำหนดจุด**  
จะต้องทำผ่านการโหลดmodel โตยตรงแต่ไม่สามารถใช้ pipeline ได้

In [ ]:
from transformers import SamModel, SamProcessor

model = SamModel.from_pretrained("Zigeng/SlimSAM-uniform-77").to("cuda")
processor = SamProcessor.from_pretrained("Zigeng/SlimSAM-uniform-77")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# recheck alread cuda work

In [ ]:
input_points = [[[250, 450]]] # format be list of list of list
inputs = processor(image,input_points=input_points, return_tensors="pt").to(device)
# change inputs from cpu to gpu before processing
# return type be pytorch sensors

In [ ]:
import torch
# torch no grad ใช้บอกเพื่อจะใข้งาน model โดยไม่มีการ update model ในช่วงการใช้งาน
with torch.no_grad():
  output = model(**inputs) # format be two *
predicted_masks = processor.image_processor.post_process_masks(
    output.pred_masks,
    inputs['original_sizes'],
    inputs['reshaped_input_sizes']
)

In [ ]:
predicted_masks[0].shape
# 3 สื่อถึงมี 3 candidate masks, 2 ตัวหลังคือ high width

In [ ]:
output.iou_scores
# สื่อถึงความมันใจของแต่ละ mask มีค่าเท่าไหร

In [ ]:
def show_mask_on_image(raw_image, mask):
  if not isinstance(mask, torch.Tensor):
    masks = torch.Tensor(mask)
  if len(mask.shape) == 4:
    mask = mask.squeeze()
    # บาง model จะมี shape เป็นสีตัวก็ใช้ squeeze เพื่อลด มิติให้เหลือสามมิติโดยการลบตัวแรกออกไป
  print(mask.shape)
  fig, axes = plt.subplots(1,1, figsize=(15,15))
  mask = mask.cpu().detach()
  # ย้าย mask จาก tensor มาที่ cpu แล้วแยก tensor จากการคำนวณออกมา
  axes.imshow(np.array(raw_image))
  show_mask(mask,axes)
  plt.show()

In [ ]:
for i in range(len(predicted_masks[0][:])):
  show_mask_on_image(image, predicted_masks[0][:,i])

**Depth Estimation**  
คือการประมาณความลึกจากภาพ เช่น Model DPT-Hybrid

In [ ]:
from transformers import pipeline

In [ ]:
model = 'Intel/dpt-hybrid-midas'
depth_estimator = pipeline('depth-estimation', model=model)

In [ ]:
result_depth = depth_estimator(image)

In [ ]:
result_depth

In [ ]:
result_depth.keys()

In [ ]:
plt.imshow(result_depth['depth'])
plt.colorbar(label='Depth')
plt.show()
# ยิ่งใกล้จะมีค่ามาก ยิ่งไกลจะน้อย